### 0. Imports

In [228]:
import pandas as pd
import yfinance as yf
import numpy as np
import xlrd
from itertools import combinations

### 1. Data download

In [148]:
# read ticker data as Series and convert to list
tickers = pd.read_csv('tickers.csv', squeeze=True)
tickers = tickers.tolist()

In [149]:
price_list = []

for ticker in tickers:
    stock = yf.Ticker(ticker).history(period="3mo").reset_index()
    stock = stock[['Date', 'Close']]
    stock.rename(columns={'Close': ticker}, inplace=True)
    price_list.append(stock)

- BFB: No data found, symbol may be delisted


### 2. Compute ratios

In [261]:
# Concatenate all prices into one big dataframe
df = pd.concat(price_list, axis=1)
df = df.loc[:,~df.columns.duplicated()].copy()

In [ ]:
#file = pd.concat(pd.read_excel("path to excel file on the local computer", sheet_name=None, skiprows=0, \
#                               index_col=0), axis=1, sort=False)

#file.columns = file.columns.droplevel(level=1)
file_new = pd.DataFrame(file.drop(labels="Date"))
file_new.rename_axis("Date", axis=1)

df_new = pd.concat([file_new[file_new.columns.difference([col])].div(file_new.where(file_new !=0, np.nan)[col], axis=0)\
                           .add_suffix("_" + col) for col in file_new.columns], axis=1)

df_new.rename_axis("Date", axis=1)
df_delta = df_new.pct_change(periods=-1)
df_delta = df_delta.iloc[:4]
df_newstocks = df_delta[df_delta.iloc[:] >= 0]
positiveRatiosPercent = df_newstocks.drop(df_newstocks.columns[df_newstocks.apply(lambda col: col.isnull().sum() > 0)], axis=1)
labelList = list(positiveRatiosPercent.columns.values)
df_positiveRatios = df_new[labelList]
df_positiveRatios.rename(columns = {"Unnamed: 0" : "Dates"}, inplace = True)
df_positiveRatios.to_excel("path to excel file on the local computer")